In [8]:
#--------------------------------------------------------------
#-----------------データ作成フェーズ---------------------------
#--------------------------------------------------------------
TRAIN_NUM=10000
TEST_NUM =10000
ROUND=int(input("Input the round number : "))
gpu_num=input("Input the gpu number : ")

from present import *
cp=Present(0,ROUND)

import time
start=time.time()
import numpy as np

import sys
#暗号器モジュールにバグがないかのチェック
if(cp.test()):
    print("Successfully generated a instance of Cryptographic machine!")
else:
    print("Exit the session...")
    sys.exit(1)


#入力差分(wang)
delta_plain=[
    0x7000000000007000,
    0x0700000000000700,
    0x0070000000000070,
    0x0007000000000007
]

from Crypto.Random import get_random_bytes

#-----------------データ保管場所----------------------------------
import os
path = "./Datas"
os.makedirs("./Datas", exist_ok=True)#-------結果保存用--------

x_train=open("./Datas/x_train.txt","w")
y_train=open("./Datas/y_train.txt","w")
x_test=open("./Datas/x_test.txt","w")
y_test=open("./Datas/y_test.txt","w")


#-----------------------データ作成開始------------------------------
SUM=TRAIN_NUM+TEST_NUM
for i in range(SUM): 
    
    #鍵，平文をランダムに設定
    mas_key_char  =get_random_bytes(10)#10 byte 文字列
    mas_key=int.from_bytes(mas_key_char,byteorder='big')
    #print("mas_key : 0x"+format(mas_key,"020x"))#-----------------------test------
    plain_char=get_random_bytes(8)#8 byte 文字列
    plain=int.from_bytes(plain_char,byteorder='big')
    #print("plain : 0x"+format(plain,"016x"))#-----------------------test------
    
    #暗号文
    cp = Present(mas_key, ROUND)
    ciph=cp.encrypt(plain)
    #print("ciph : 0x"+format(ciph,"016x"))#-----------------------test------
    
    for k in range(len(delta_plain)):
        ciph_dash=cp.encrypt(plain^delta_plain[k])#cipher_dash
        delta_ciph=ciph ^ ciph_dash#出力差分
        #print("ciph_dash : 0x"+format(ciph_dash,"016x"))#-----------------------test------
        #print("delta_ciph : 0x"+format(delta_ciph,"016x"))#-----------------------test------
        
        if(i<TRAIN_NUM):
            y_train.write(str(k))#入力差分クラス
            y_train.write("\n")
            
            for j in range(64):
                x_train.write(str((delta_ciph>>j)&1))
                x_train.write(" ")  
            x_train.write("\n")
        else:
            y_test.write(str(k))#入力差分クラス  
            y_test.write("\n")
            
            for j in range(64):
                x_test.write(str((delta_ciph>>j)&1))
                x_test.write(" ")  
            x_test.write("\n")

x_train.close()
y_train.close()
x_test.close()
y_test.close()


#--------------------------------------------------------------
#--------------------モデリング開始----------------------------
#--------------------------------------------------------------
import numpy as np

import time
start=time.time()
#---------------------配列への変換------------------------
x_train=np.loadtxt("./Datas/x_train.txt")
y_train=np.loadtxt("./Datas/y_train.txt")
x_test=np.loadtxt("./Datas/x_test.txt")
y_test=np.loadtxt("./Datas/y_test.txt")
print("Completed Data_Trainslation at %d.sec" % (int(time.time() - start)))#リストへの変換の完了メッセージ



#--------------------------------------------------------------
#----------------------モデル作成------------------------------
#--------------------------------------------------------------

#-----------------------使用するGPU指定------------------------
import os
import tensorflow as tf
# Specify which GPU(s) to use
os.environ["CUDA_VISIBLE_DEVICES"] = gpu_num  # Or 2, 3, etc. other than 0

# On CPU/GPU placement
config = tf.compat.v1.ConfigProto(allow_soft_placement=True, log_device_placement=True)
config.gpu_options.allow_growth = True
tf.compat.v1.Session(config=config)

# Note that ConfigProto disappeared in TF-2.0
    

from tensorflow import keras
model = keras.Sequential()

#中間層の追加
from tensorflow.python.keras.layers import Dense

model.add(
    Dense(
        units=128,
        input_shape=(64,),
        activation="relu",
        kernel_initializer=keras.initializers.RandomNormal(mean=0.0, stddev=0.05, seed=None)
    )
)
model.add(
    Dense(
        units=1024,
        activation="relu",
        kernel_initializer=keras.initializers.RandomNormal(mean=0.0, stddev=0.05, seed=None)
    )
)
#出力層の追加
model.add(
    Dense(
        units=4,
        activation='softmax',#シグモイドの拡張 出力値の和が１になる関数
        kernel_initializer=keras.initializers.RandomNormal(mean=0.0, stddev=0.05, seed=None)
    )
)


#---------------------------------学習-------------------------------
#from tensorflow.python.keras.callbacks import TensorBoard
from tensorflow.python.keras.utils import to_categorical

# Early-stopping 
#early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=100, verbose=0, mode='auto')#この形式で書かないと怒られる
opt = keras.optimizers.Adam(learning_rate=0.001)#オプティマイザー
model.compile(
    optimizer=opt,
    #loss='categorical_crossentropy',
    loss='mse',
    metrics=['accuracy']
)

#tsb=TensorBoard(log_dir='./logs')
history_adam=model.fit(
    x_train,#x_train 
    to_categorical(y_train,4),#y_train のiビット目の列を抽出 の，２でカテゴリ化したやつ
    batch_size=200,
    epochs=25,
    #nb_epoch=1000,
    validation_split=0.3,
    #callbacks=[tsb]
    #callbacks=[early_stopping]#アーリーストッピング
)

#全作業終了時に時間を表示
print("All operations has completed at %d.sec" % (int(time.time() - start)))

Input the round number : 2
Input the gpu number : 2
Successfully generated a instance of Cryptographic machine!
Completed Data_Trainslation at 5.sec
Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:0 -> device: XLA_GPU device
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: GeForce GTX TITAN Black, pci bus id: 0000:05:00.0, compute capability: 3.5

Train on 28000 samples, validate on 12000 samples
Epoch 1/25
28000/28000 [==============================] - 1s 29us/sample - loss: 0.1126 - acc: 0.6481 - val_loss: 0.0840 - val_acc: 0.7428
Epoch 2/25
28000/28000 [==============================] - 1s 24us/sample - loss: 0.0748 - acc: 0.7786 - val_loss: 0.0664 - val_acc: 0.8133
Epoch 3/25
28000/28000 [==============================] - 1s 25us/sample - loss: 0.0601 - acc: 0.8299 - val_loss: 0.0594 - val_acc: 0.8216
Epoch 4/25
28000/28000 [==============================] - 1s 24us/sample - lo

In [6]:
#---------------------------------------Testing Phase---------------------
predict=model.predict(x_test)#テスト 結果をpredictに
print(predict)

[[1.00000000e+00 2.36285241e-10 2.40069409e-10 2.20499863e-09]
 [1.90606748e-08 1.00000000e+00 9.03092271e-12 2.29013374e-21]
 [6.78752258e-05 6.40085936e-01 3.59747708e-01 9.84237995e-05]
 ...
 [1.39108375e-02 9.62769091e-01 4.76643397e-03 1.85536388e-02]
 [4.22752237e-06 3.93990602e-04 9.99601781e-01 8.02304334e-11]
 [1.45132110e-01 4.19729804e-05 8.26543012e-09 8.54825974e-01]]


In [7]:
model.evaluate(x_test, to_categorical(y_test,4))

40000/40000 [==============================] - 3s 69us/sample - loss: 0.0486 - acc: 0.8617


[0.04859025435931981, 0.8617]